# Implémentation de méthodes élémentaires pour la classification supervisée : Naive Bayes et classifieur par plus proches voisins

Pour ce TP, nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix

Le jeu de données [Vertebral Column](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column) permet d'étudier les pathologies d'hernie discale et de Spondylolisthesis. Ces deux pathologies sont regroupées dans le jeu de données en une seule catégorie dite `Abnormale`. 

Il s'agit donc d'un problème de classification supervisée à deux classes :
- Normale (NO) 
- Abnormale (AB)    

avec 6 variables bio-mécaniques disponibles (features).

L'objectif du TP est d'implémenter quelques méthodes simples de classification supervisée pour ce problème.

# Importation des données

> Télécharger le fichier column_2C.dat depuis le site de l'UCI à [cette adresse](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column). 
>
> On peut importer les données sous python par exemple avec la librairie [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html). Vous pourrez au besoin consulter la documentation de la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). 
> 
> Le chemin donné dans la fonction `read_csv`est une chaîne de caractère qui spécifie le chemin complet vers le ficher sur votre machine. On peut aussi donner une adresse url si le fichier est disponible en ligne.
>
> Attention à la syntaxe pour les chemins sous Windows doit etre de la forme  `C:/truc/machin.csv`. 
> 
> Voir ce [blog](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) pour en savoir plus sur la "manipulation des chemins" sur des OS variés. 

In [2]:
file_path= 'column_2C.dat'
Vertebral = pd.read_csv(file_path,
                          delim_whitespace= ',',
                          header= None)
Vertebral.columns = ["pelvic_incidence",
                     "pelvic_tilt",
                     "lumbar_lordosis_angle",
                     "sacral_slope",
                     "pelvic_radius",
                     "degree_spondylolisthesis",
                     "class"]

> Vérifier à l'aide des méthodes `.head()`  et `describe()` que les données sont bien importées.

In [3]:
Vertebral.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.03,22.55,39.61,40.48,98.67,-0.25,AB
1,39.06,10.06,25.02,29.00,114.41,4.56,AB
2,68.83,22.22,50.09,46.61,105.99,-3.53,AB
3,69.30,24.65,44.31,44.64,101.87,11.21,AB
4,49.71,9.65,28.32,40.06,108.17,7.92,AB


In [4]:
Vertebral.describe()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,60.496484,17.542903,51.930710,42.953871,117.920548,26.296742
std,17.236109,10.008140,18.553766,13.422748,13.317629,37.558883
min,26.150000,-6.550000,14.000000,13.370000,70.080000,-11.060000
25%,46.432500,10.667500,37.000000,33.347500,110.710000,1.600000
50%,58.690000,16.360000,49.565000,42.405000,118.265000,11.765000
75%,72.880000,22.120000,63.000000,52.692500,125.467500,41.285000
max,129.830000,49.430000,125.740000,121.430000,163.070000,418.540000


In [5]:
Vertebral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pelvic_incidence          310 non-null    float64
 1   pelvic_tilt               310 non-null    float64
 2   lumbar_lordosis_angle     310 non-null    float64
 3   sacral_slope              310 non-null    float64
 4   pelvic_radius             310 non-null    float64
 5   degree_spondylolisthesis  310 non-null    float64
 6   class                     310 non-null    object 
dtypes: float64(6), object(1)
memory usage: 17.1+ KB


> Les librairies de Machine Learning telles que `sckitlearn` prennent en entrée des tableau numpy (pas des objets pandas). Créer un tableau numpy que vous nommerez `VertebralVar` pour les features et un vecteur numpy `VertebralClas` pour la variable de classe. Voir par exemple [ici](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy).

In [6]:
Vertebral.iloc[198:205]

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
198,74.85,13.91,62.69,60.95,115.21,33.17,AB
199,75.30,16.67,61.30,58.63,118.88,31.58,AB
200,63.36,20.02,67.50,43.34,131.00,37.56,AB
201,67.51,33.28,96.28,34.24,145.60,88.30,AB
202,76.31,41.93,93.28,34.38,132.27,101.22,AB
203,73.64,9.71,63.00,63.92,98.73,26.98,AB
204,56.54,14.38,44.99,42.16,101.72,25.77,AB


In [7]:
VertebralVar = Vertebral.drop(columns='class')
VertebralVar = VertebralVar.to_numpy()

# Printing the NumPy array
#print(VertebralVar[100:300,0:-1])
#print(VertebralVar[0:200,0:-1])
#print(VertebralVar[50:250,0:-1])
print(VertebralVar[50:251,0:-1])

[[ 55.29  20.44  34.    34.85 115.88]
 [ 74.43  41.56  27.7   32.88 107.95]
 [ 50.21  29.76  36.1   20.45 128.29]
 ...
 [ 47.32   8.57  35.56  38.75 120.58]
 [ 50.75  20.24  37.    30.52 122.34]
 [ 36.16  -0.81  33.63  36.97 135.94]]


In [8]:
VertebralClas = Vertebral['class'].to_numpy()

VertebralClas

array(['AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'A

# Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 60% des observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de test. 

In [9]:
from sklearn.model_selection import train_test_split
VertebralVar_train,VertebralVar_test,VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar, VertebralClas, train_size = 0.6)
ntot = len(VertebralClas) ### longueur totale de l'échantillon -  TO DO ####
ntrain = len(VertebralClas_train) ### longueur totale de l'échantillon d'apprentissage - TO DO ####
ntest = len(VertebralClas_test)### longueur totale de l'échantillon de test -TO DO ####

Remarque : on peut aussi le faire à la main avec la fonction [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

# Extraction des deux classes

> Extraire les deux sous-échantillons de classes respectives "Abnormale" et "Normale" pour les données d'apprentissage et de test.

In [10]:
VertebralVar_train_AB = VertebralVar_train[VertebralClas_train == 'AB']
VertebralVar_train_NO = VertebralVar_train[VertebralClas_train == 'NO']
print(VertebralVar_train_AB)

[[ 8.5100e+01  2.1070e+01  9.1730e+01  6.4030e+01  1.0906e+02  3.8030e+01]
 [ 9.5380e+01  2.4820e+01  9.5160e+01  7.0560e+01  8.9310e+01  5.7660e+01]
 [ 5.8780e+01  7.6700e+00  5.3340e+01  5.1120e+01  9.8500e+01  5.1580e+01]
 [ 8.5640e+01  4.2690e+01  7.8750e+01  4.2950e+01  1.0514e+02  4.2890e+01]
 [ 7.7110e+01  3.0470e+01  6.9480e+01  4.6640e+01  1.1215e+02  7.0760e+01]
 [ 5.7300e+01  2.4190e+01  4.7000e+01  3.3110e+01  1.1681e+02  5.7700e+00]
 [ 6.9560e+01  1.5400e+01  7.4440e+01  5.4160e+01  1.0507e+02  2.9700e+01]
 [ 3.1280e+01  3.1400e+00  3.2560e+01  2.8130e+01  1.2901e+02  3.6200e+00]
 [ 8.9680e+01  3.2700e+01  8.3130e+01  5.6980e+01  1.2996e+02  9.2030e+01]
 [ 5.7520e+01  3.3650e+01  5.0910e+01  2.3880e+01  1.4098e+02  1.4875e+02]
 [ 4.3920e+01  1.4180e+01  3.7830e+01  2.9740e+01  1.3446e+02  6.4500e+00]
 [ 7.5300e+01  1.6670e+01  6.1300e+01  5.8630e+01  1.1888e+02  3.1580e+01]
 [ 8.1110e+01  2.0690e+01  6.0690e+01  6.0420e+01  9.4020e+01  4.0510e+01]
 [ 7.1190e+01  2.3900e+01

In [11]:
n_AB = len(VertebralVar_train_AB)
n_NO = len(VertebralVar_train_NO)
print(n_AB)
print(n_NO)

125
61


# Gaussian Naive Bayes

Nous allons ajuster un classifieur naif bayesien sur les données d'apprentissage.

Pour une observation $x \in \mathbb R^6$, la régle du MAP consiste à choisir la catégorie $\hat y (x) = \hat k $ qui maximise (en $k$) 
$$ score_k(x) = \hat \pi_k \prod_{j=1} ^6  \hat f_{k,j}(x_j)   $$
où :
- $k$ est le numéro de la classe ;
- $\hat \pi_k$ est la proportion observée de la classe $k$, 
- $\hat f_{k,j} $ est la densité gaussienne univariée de la classe $k$ pour la variable $j$. Les paramètres de cette loi valent (ajustés par maximum de vraisemblance) :
    - $\hat \mu_{k,j}$ : la moyenne empirique de la variable $X^j$ restreinte à la classe k,
    - $ \hat \sigma^2_{k,j}$ : la variance empirique de la variable $X^j$ restreinte à la classe k.
    
Noter que la fonction $x \mapsto  \prod_{j=1} ^6  f_{k,j}(x_j) $ peut aussi être vue comme une densité gaussienne multidimensionnelle de moyenne $(\mu_{k,1}, \dots, \mu_{k,6})$ et de matrice de covariance diagonale $diag(\hat \sigma^2_{k,1},\dots,\hat  \sigma^2_{k,6})$. Cette remarque évite de devoir calculer le produit de 6 densités univariées, à la place on calcule plus directement la valeur de la densité multidimensionnelle.

Pour calculer la valeur de la densité d'une gaussienne multidimensionnelle en un point $x$ de $\mathbb R ^d$ on peut utililser la fonction [`multivariate_normal`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) de la librairie [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html). 

On pourra utiliser la fonction `var` de numpy pour calculer le vecteur des variances.

Calcul des moyennes et des variances de chaque variable pour chacun des deux groupes :

In [12]:
mean_AB = VertebralVar_train_AB.mean(axis = 0) 
mean_NO = VertebralVar_train_NO.mean(axis = 0) 

# variances estimées variable par variable pour AB (sur le train) :
var_AB = VertebralVar_train_AB.var(axis = 0) 
# variances estimées variable par variable pour NO (sur le train) :
var_NO = VertebralVar_train_NO.var(axis = 0)

# on forme les matrices de covariance (matrices diagonales car indep) :
Cov_NB_AB = np.diag(var_AB)
Cov_NB_NO = np.diag(var_NO)


Calcul du "score" sur chaque groupe pour chaque element des données test : 

In [13]:
gnb_AB = multivariate_normal(mean=mean_AB, cov=Cov_NB_AB)
gnb_NO = multivariate_normal(mean=mean_NO, cov=Cov_NB_NO)

likelihood_AB = gnb_AB.pdf(VertebralVar_test)
likelihood_NO = gnb_NO.pdf(VertebralVar_test)

prior_AB = n_AB / (n_AB + n_NO)
prior_NO = 1 - prior_AB

score_AB = likelihood_AB * prior_AB
score_NO = likelihood_NO * prior_NO

Vertebral_Clas_pred = np.where(score_AB > score_NO, 'AB', 'NO')

Vertebral_Clas_pred

array(['NO', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'NO', 'NO',
       'AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO',
       'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB',
       'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'NO', 'AB',
       'NO', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'NO',
       'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB',
       'NO', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO',
       'NO', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'AB', 'NO',
       'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB',
       'NO', 'NO', 'NO', 'NO', 'NO', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB',
       'AB', 'AB', 'NO'], dtype='<U2')

La matrice de confusion est une matrice qui synthétise les performances d'une régle de classification. Chaque ligne correspond à une classe réelle, chaque colonne correspond à une classe estimée. La cellule (ligne L, colonne C) contient le nombre d'éléments de la classe réelle L qui ont été estimés comme appartenant à la classe C. Voir par exemple [ici](https://fr.wikipedia.org/wiki/Matrice_de_confusion).

> Evaluer les performances de la méthode sur l'échantillon test. Vous pourrez utiliser la fonction [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) de la librairie [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [14]:
cnf_matrix_test = confusion_matrix(y_true = VertebralClas_test, y_pred = Vertebral_Clas_pred)
cnf_matrix_test#.astype('float') / cnf_matrix_test.sum(axis=1).reshape(-1,1) 


array([[62, 23],
       [ 1, 38]], dtype=int64)

>  Il existe bien sûr une fonction scikit-learn  pour la méthode Naive Bayes : voir [ici](http://scikit-learn.org/stable/modules/naive_bayes.html). Vérifier que votre prédicteur donne la même réponse de cette fonction.

In [15]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X = VertebralVar_train, y = VertebralClas_train)
y_predicted = gnb.predict(X = VertebralVar_test)
confusion_matrix(y_true = VertebralClas_test, y_pred = y_predicted)

array([[62, 23],
       [ 1, 38]], dtype=int64)

# Classifieur par plus proches voisins

Il est préférable d'utiliser la structure de données de type [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) pour effectuer des requêtes de plus proches voisins dans un nuage de points. 

> Contruction du k-d tree pour les données train (pour la métrique euclidienne) :

In [16]:
from sklearn.neighbors import KDTree
tree =  KDTree(VertebralVar_train, metric = 'euclidean')

> Rechercher les 10 plus proches voisins dans les données d'apprentissage du premier point des données de test et afficher les classes de ces observations voisines.

In [17]:
distances, indices_voisins = tree.query(VertebralVar_test[0].reshape(1,-1), k = 10)
classes_voisins = VertebralClas_train[indices_voisins]
print(classes_voisins)    

[['NO' 'AB' 'AB' 'NO' 'AB' 'AB' 'AB' 'NO' 'NO' 'AB']]


In [21]:
indices_voisins.shape

(124, 5)

In [23]:
VertebralClas_train.shape

(186,)

In [24]:
classes_voisins.shape

(124, 5)

In [18]:
VertebralClas_train.shape

(186,)

Pour le classifieur par plus proches vosins, la prediction est la classe majoritaire des k plus proches voisins.

> Donner la prédiction pour le premier point de test par vote majoritaire sur ses 10 plus proches voisins 

In [19]:
# Calculate the mode
pd.DataFrame(classes_voisins).mode(axis = 1)

,0
0,AB


> Donner la prediction du classifieur ppv pour toutes les données de test. Evaluer la qualité du classifieur.

In [20]:
k_class = 5
distances, indices_voisins = tree.query(VertebralVar_test, k = k_class)
classes_voisins = VertebralClas_train[indices_voisins]

pred_kNN_test = pd.DataFrame(classes_voisins).mode(axis = 1).loc[:,0]
pred_kNN_test = pred_kNN_test.to_numpy()

cnf_matrix_kNN = confusion_matrix(pred_kNN_test, VertebralClas_test)
cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1).reshape(-1,1) 

array([[0.87777778, 0.12222222],
       [0.17647059, 0.82352941]])

Il existe bien sûr une fonction scikit-learn pour le classifieur plus proche voisin, voir [ici](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).